In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('incident_event_log.csv')

In [3]:
#change all ? marks to NAN values so we can manipulate them easier
df_nan = df.copy()
df_nan = df.replace('?',np.nan)

In [4]:
#changing all objects to datetime values in the date calumns that we will use for our target columns
df_nan['opened_at']=pd.to_datetime(df_nan['opened_at'],dayfirst= True)
df_nan['resolved_at'] = pd.to_datetime(df_nan['resolved_at'], dayfirst = True)

In [5]:
# creating a difference column to show the amount of days it takes to resolve a ticket that will be used for the target column
df_nan['difference'] = (df_nan['resolved_at'] - df_nan['opened_at']).dt.days

In [6]:
#creates a target column of 0s and 1s based on the amount of days it took to resolve the incident.
#if the amount of days is greater than 2 or NAN it turns to 1 and if 2 or below it is a 0
df_nan['target_colum'] = df_nan.apply(lambda row: 1 if pd.notna(row['resolved_at'])
                                       and (row['resolved_at'] - row['opened_at']).days > 2 else 0, axis = 1)

In [ ]:
columns_with_nan = df_nan.columns[df_nan.isna().any()].tolist()
columns_with_nan

In [ ]:
# Find the percentage of null values in each column
df_nan.isna().sum()/len(df_nan)

In [ ]:
# Creating a function to prioritize 'new' and 'active' incident states that will emliminate
"""Parmerter: takes grouped data frame and applys to each group of data
returns a data frame that only includes one instance of every value in the 'number' column
that has a correlating 'incident_state' value of 'New' if possible and then filters to 'active', 
while also including one instance of a 'number' that does not have new or active in 'incident_state' """
def new_active_incident_state(group):
    if (group['incident_state'] == 'New').any():
        return group[group['incident_state'] == 'New'].iloc[0]
    elif (group['incident_state'] == 'Active').any():
        return group[group['incident_state'] == 'Active'].iloc[0]
    else:
        return group.iloc[0]  # In case neither 'new' nor 'active' exists, retain the first occurrence

# Apply this function to the DataFrame by grouping by 'number'
df_new_active = df_nan.groupby('number',group_keys=False).apply(new_active_incident_state).reset_index(drop=True)
# View the result
df_new_active

###warning note: group_keys = Flase is not yet apart of pandas and thus will not work, however by including group_keys = False exlcudes the 'number' column form being included in the apply results and should avoid altering the sturcture of the result


## create a data frame with the following columns for training
#List of columns to keep for Training set:
* opened_by: identifier of the user who reported the incident
* opened_at: incident user opening date and time
* sys_created_by: identifier of the user who registered the incident
* sys_created_at: incident system creation date and time
* contact_type: categorical attribute that shows by what means the incident was reported
* location: identifier of the location of the place affected
* category: first-level description of the affected service
* subcategory: second-level description of the affected service (related to the first level description, i.e., to category);
* u_symptom: description of the user perception about service availability
* cmdb_ci: (confirmation item) identifier used to report the affected item (not mandatory)
* impact: description of the impact caused by the incident (values: 1â€“High; 2â€“Medium; 3â€“Low);
* urgency: description of the urgency informed by the user for the incident resolution (values: 1â€“High; 2â€“Medium; 3â€“Low);
* priority: calculated by the system based on 'impact' and 'urgency';
* assignment_group: identifier of the support group in charge of the incident;
* assigned_to: identifier of the user in charge of the incident;
* knowledge: boolean attribute that shows whether a knowledge base document was used to resolve the incident;
* u_priority_confirmation: boolean attribute that shows whether the priority field has been double-checked;
* notify: categorical attribute that shows whether notifications were generated for the incident;
* problem_id: identifier of the problem associated with the incident;
* rfc: (request for change) identifier of the change request associated with the incident;


* vendor:(probably drop only 15 entries) identifier of the vendor in charge of the incident;
* caused_by:(drop only 2 entires) identifier of the RFC responsible by the incident


list of potential:exclude these columns
* sys_updated_by:(? is this generated day 1?) identifier of the user who updated the incident and generated the current log record
* sys_updated_at:(?day 1 info?) incident system update date and time


**Who are we presenting too:** ServiceNow IT department

#### Target
* create target column: numbers of days ticket is open???
* what could this tell us, or how could we explain this as usefull: if we can succesfully predict whether an incident will be resolved in sufficent time given info porovided about the incident. then we could advise...???
* who wants this

#### TO DO List
## Day 1 goal: 
* **complete**create repository and seperate branches  link: https://github.com/Sug-ar-N-Spice/Incident_Events.git
* find the NANS and figure out how to fill
* **complete**turn all ? into NANs
* **complete**find documentation/info of columns   link: https://archive.ics.uci.edu/dataset/498/incident+management+process+enriched+event+log
* **semi-complete**create target column: complete for tickets open over 2 days
* decide if the target column is relavant enough for conclusion
* decide if there are any columns we can get drop ..ex.'number'
* also decide what columns would cuase dataleaks to give the model all the answers renderring our model useless..ex. time/date, days open, other columns?
* list out the models that we will attempt to use find code examples/syntax ready
* Parameter loops
* do we need fucntions? because this is our only dataset, we don't have another one to test?
* **complete** list all columns with NaN values and then figure out how to fill them
* DO we need a standard scaler
* DO we need PCA to reduce the columns
* Do we need resampler
* clustering? dont we need it for certain models? #original task is regresion and clustering should we do this to understand data

**curenlty working on:**
  Figure out how to filter to find all duplicate incedents for same value in 'number' and only retain 1 incedence set to 'incident_status == 'new'


In [ ]:
#this shows a balanced dataset for the target column
#however need to check on further filtered data frame for balance
df_nan['target_colum'].value_counts()

In [14]:
#This shows that our filtered dataFrame retains all of the unique values in the 'number' column
duplicates_list = df_nan['number'].unique().tolist()
display(len(duplicates_list))
unique_list_check = df_new_active['number'].unique().tolist()
display(len(unique_list_check))

24918

24918

In [13]:
#shows the total 8 possible values for filtering compared to the output of our filtered data frame
display(df_nan.value_counts('incident_state'))
display(df_new_active.value_counts('incident_state'))


incident_state
Active                38716
New                   36407
Resolved              25751
Closed                24985
Awaiting User Info    14642
Awaiting Vendor         707
Awaiting Problem        461
Awaiting Evidence        38
-100                      5
Name: count, dtype: int64

incident_state
New                   16397
Active                 6566
Resolved               1816
Awaiting User Info      133
Awaiting Problem          6
Name: count, dtype: int64

In [79]:
# Awaiting User Info      133
# Awaiting Problem          6
df_awaiting = df_new_active.loc[
    #(df_new_active['incident_state'] == 'Awaiting User Info') | 
    (df_new_active['incident_state'] == 'Awaiting Problem')]
df_awaiting

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at,difference,target_colum
867,INC0001402,Awaiting Problem,True,0,0,0,True,Caller 1498,Opened by 468,2016-03-02 15:09:00,...,NaN,NaN,NaN,NaN,code 8,Resolved by 69,2016-03-04 10:54:00,9/3/2016 11:00,1.0,0
3188,INC0005033,Awaiting Problem,True,0,0,0,True,Caller 5466,Opened by 423,2016-03-09 15:57:00,...,NaN,NaN,NaN,NaN,code 6,Resolved by 182,2016-04-27 13:32:00,2/5/2016 14:07,48.0,1
3223,INC0005092,Awaiting Problem,True,0,0,0,True,Caller 4875,Opened by 423,2016-03-09 18:03:00,...,NaN,NaN,NaN,NaN,code 6,Resolved by 182,2016-05-06 11:53:00,11/5/2016 12:07,57.0,1
3341,INC0005298,Awaiting Problem,True,0,0,0,True,Caller 3898,Opened by 423,2016-03-10 10:34:00,...,NaN,NaN,NaN,NaN,code 6,Resolved by 182,2016-04-26 11:55:00,1/5/2016 12:07,47.0,1
3343,INC0005303,Awaiting Problem,True,0,0,0,True,Caller 5466,Opened by 423,2016-03-10 10:37:00,...,NaN,NaN,NaN,NaN,code 6,Resolved by 182,2016-05-11 11:36:00,16/5/2016 12:07,62.0,1
7574,INC0011584,Awaiting Problem,True,0,0,0,True,Caller 2708,Opened by 17,2016-03-27 14:27:00,...,NaN,NaN,NaN,NaN,code 6,Resolved by 11,2016-03-27 14:55:00,1/4/2016 14:59,0.0,0


In [28]:
df_awaiting_problem = df_new_active.loc[df_new_active['made_sla'] == False]
df_awaiting_problem

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at,difference,target_colum
1630,INC0002588,Active,True,0,0,0,False,Caller 3571,Opened by 325,2016-03-04 15:00:00,...,NaN,NaN,NaN,NaN,code 1,Resolved by 143,2016-03-11 10:38:00,24/3/2016 18:50,6.0,1


In [ ]:
df_awaiting_problem = df_new_active.loc[df_new_active['u_priority_confirmation'] == True]
df_awaiting_problem.head()

In [48]:
df_new_active['priority'].unique()  # Check the unique values in the 'priority' column

array(['3 - Moderate', '4 - Low', '1 - Critical', '2 - High'],
      dtype=object)

In [ ]:
df_priotity = df_new_active.loc[df_new_active['priority'] == '1 - Critical']
df_priotity



In [ ]:
df_new_active.info()

In [63]:
df.columns

Index(['number', 'incident_state', 'active', 'reassignment_count',
       'reopen_count', 'sys_mod_count', 'made_sla', 'caller_id', 'opened_by',
       'opened_at', 'sys_created_by', 'sys_created_at', 'sys_updated_by',
       'sys_updated_at', 'contact_type', 'location', 'category', 'subcategory',
       'u_symptom', 'cmdb_ci', 'impact', 'urgency', 'priority',
       'assignment_group', 'assigned_to', 'knowledge',
       'u_priority_confirmation', 'notify', 'problem_id', 'rfc', 'vendor',
       'caused_by', 'closed_code', 'resolved_by', 'resolved_at', 'closed_at'],
      dtype='object')

In [ ]:
df_closed_code = df_new_active.loc[df_new_active['closed_code'] == 'code 7']
df_closed_code

In [73]:
df_new_active[['opened_at', 'sys_updated_at']]

,opened_at,sys_updated_at
0,2016-02-29 01:16:00,29/2/2016 01:23
1,2016-02-29 04:40:00,29/2/2016 04:57
2,2016-02-29 06:10:00,29/2/2016 06:26
3,2016-02-29 06:38:00,29/2/2016 06:42
4,2016-02-29 06:58:00,29/2/2016 07:26
...,...,...
24913,2017-02-15 02:02:00,15/2/2017 02:02
24914,2017-02-15 07:09:00,15/2/2017 07:09
24915,2017-02-15 11:58:00,15/2/2017 11:58
24916,2017-02-16 09:09:00,16/2/2017 09:09


In [72]:
df_new_active.value_counts('sys_updated_at')

sys_updated_at
7/3/2016 09:11    7
7/3/2016 10:20    6
6/3/2016 10:52    6
4/5/2016 11:45    5
7/3/2016 13:49    5
                 ..
2/3/2016 10:18    1
2/3/2016 10:20    1
2/3/2016 10:25    1
2/3/2016 10:27    1
9/8/2016 16:51    1
Name: count, Length: 19562, dtype: int64

In [67]:
df_new_active['closed_code'].isna().count()

24918

In [65]:
df_new_active.value_counts('closed_code')

closed_code
code 6     14882
code 7      4412
code 9      2489
code 8       831
code 5       677
code 1       449
code 10      314
code 11      221
code 16      171
code 4       130
code 3        99
code 2        59
code 15       43
code 17       21
code 13        8
code 14        3
code 12        2
Name: count, dtype: int64

In [25]:

df_new_active.value_counts('made_sla')
#false not from Awaiting Problems

made_sla
True     24917
False        1
Name: count, dtype: int64

In [17]:
df_new_active.value_counts('reopen_count')
#not from awaiting problems

reopen_count
0    24915
1        3
Name: count, dtype: int64

In [116]:
df_inc = df.loc[df['number'] == 'INC0011551']
df_inc

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,sys_created_by,sys_created_at,sys_updated_by,sys_updated_at,contact_type,location,category,subcategory,u_symptom,cmdb_ci,impact,urgency,priority,assignment_group,assigned_to,knowledge,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at
50379,INC0011551,Resolved,True,0,0,0,True,Caller 5589,Opened by 17,27/3/2016 08:30,?,?,Updated by 908,27/3/2016 08:30,Phone,Location 204,Category 32,Subcategory 174,Symptom 491,?,2 - Medium,2 - Medium,3 - Moderate,Group 70,Resolver 13,False,False,Do Not Notify,?,?,?,?,code 7,Resolved by 11,?,1/4/2016 08:59
50380,INC0011551,Closed,False,0,0,1,True,Caller 5589,Opened by 17,27/3/2016 08:30,?,?,Updated by 908,1/4/2016 08:59,Phone,Location 204,Category 32,Subcategory 174,Symptom 491,?,2 - Medium,2 - Medium,3 - Moderate,Group 70,Resolver 13,False,False,Do Not Notify,?,?,?,?,code 7,Resolved by 11,?,1/4/2016 08:59


In [120]:
df_inc0000093 = df_nan.loc[df_nan['number'] == 'INC0000093']
df_inc0000093
""" 'sys_updated_at' , sys_mod_count, & 'active' are the different columns"""

" 'sys_updated_at' , sys_mod_count& 'active' are the different columns"

In [108]:
df_resolved = df_new_active.loc[df_new_active['incident_state'] == 'Resolved']
df_resolved.value_counts('number')

number
INC0000093    1
INC0018918    1
INC0019381    1
INC0019377    1
INC0019365    1
             ..
INC0011581    1
INC0011551    1
INC0011533    1
INC0011527    1
INC0034431    1
Name: count, Length: 1816, dtype: int64

In [17]:
#create empty dictionary to store stats tables for all nan columns
nan_column_stats = {}

#loop through each column in the list of columns with nan values
for col in columns_with_nan:
    #using the loc function to list the rows with Nan values in the particular column and calculate descpritive stats for all columns
    nan_column_stats[col] = df_nan.loc[df_nan[col].isna()].describe(include='all')
    pd.set_option('display.max_columns', None)

In [ ]:
#display the stats for a specific column
#pd.set_option('display.max_columns', None)
problem_id_nan_stats = nan_column_stats['problem_id']
problem_id_nan_stats

In [102]:
# Filter the DataFrame where "incident_state" is equal to "new" and "active" is equal to 0
filtered_df = df_nan[(df_nan["incident_state"] == "Resolved") & (df_nan["active"] == 'False')]

filtered_df

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,sys_created_by,sys_created_at,sys_updated_by,sys_updated_at,contact_type,location,category,subcategory,u_symptom,cmdb_ci,impact,urgency,priority,assignment_group,assigned_to,knowledge,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at,difference,target_colum


In [103]:
df.value_counts('active')

active
True     116726
False     24986
Name: count, dtype: int64

In [122]:
df_inc0019396.value_counts('resolved_at')

resolved_at
2016-05-12 23:51:00    58
Name: count, dtype: int64

In [61]:
df_inc0019396.value_counts('made_sla')

made_sla
True     57
False     1
Name: count, dtype: int64

In [59]:
df_inc0019396.value_counts('u_priority_confirmation')

u_priority_confirmation
True     56
False     2
Name: count, dtype: int64

In [105]:
# Get the value counts for 'number'
number_counts = df_nan['number'].value_counts()

# Filter for numbers that appear only once using a lambda function
only_one_number_list = number_counts[number_counts.apply(lambda x: x == 1)].index.tolist()

# View the result
only_one_number_list


###This shows that there are no values in the 'number' column that only occur once, they all have duplicate values

[]

In [74]:
df_inc0027738 = df_nan[df_nan['number'] == 'INC0027738']
df_inc0027738

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,sys_created_by,sys_created_at,sys_updated_by,sys_updated_at,contact_type,location,category,subcategory,u_symptom,cmdb_ci,impact,urgency,priority,assignment_group,assigned_to,knowledge,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at,difference,target_colum
114280,INC0027738,Active,True,0,0,0,True,Caller 3368,Opened by 131,2016-05-10 08:48:00,NaN,NaN,Updated by 908,10/5/2016 08:48,Phone,Location 93,Category 42,Subcategory 223,Symptom 534,NaN,3 - Low,3 - Low,4 - Low,Group 70,Resolver 73,False,False,Do Not Notify,NaN,NaN,NaN,NaN,code 7,Resolved by 66,2016-05-10 09:37:00,15/5/2016 10:07,0.0,0
114281,INC0027738,Resolved,True,0,0,1,True,Caller 3368,Opened by 131,2016-05-10 08:48:00,NaN,NaN,Updated by 265,10/5/2016 09:37,Phone,Location 93,Category 42,Subcategory 223,Symptom 534,NaN,3 - Low,3 - Low,4 - Low,Group 70,Resolver 73,False,True,Do Not Notify,NaN,NaN,NaN,NaN,code 7,Resolved by 66,2016-05-10 09:37:00,15/5/2016 10:07,0.0,0
114282,INC0027738,Closed,False,0,0,2,True,Caller 3368,Opened by 131,2016-05-10 08:48:00,NaN,NaN,Updated by 908,15/5/2016 10:07,Phone,Location 93,Category 42,Subcategory 223,Symptom 534,NaN,3 - Low,3 - Low,4 - Low,Group 70,Resolver 73,False,True,Do Not Notify,NaN,NaN,NaN,NaN,code 7,Resolved by 66,2016-05-10 09:37:00,15/5/2016 10:07,0.0,0


In [38]:
df_inc0019396 = df[df['number'] == 'INC0019396']
display(df_inc0019396.head())
display(df_inc0019396.tail())

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,sys_created_by,sys_created_at,sys_updated_by,sys_updated_at,contact_type,location,category,subcategory,u_symptom,cmdb_ci,impact,urgency,priority,assignment_group,assigned_to,knowledge,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at
81418,INC0019396,New,True,0,0,0,True,Caller 1580,Opened by 305,14/4/2016 20:42,?,?,Updated by 908,14/4/2016 20:42,Phone,Location 143,Category 53,Subcategory 114,?,?,2 - Medium,2 - Medium,3 - Moderate,Group 66,?,False,False,Do Not Notify,?,?,?,?,code 6,Resolved by 158,12/5/2016 23:51,18/5/2016 00:07
81419,INC0019396,New,True,0,0,1,True,Caller 1580,Opened by 305,14/4/2016 20:42,?,?,Updated by 278,14/4/2016 21:08,Phone,Location 143,Category 53,Subcategory 114,?,?,2 - Medium,2 - Medium,3 - Moderate,Group 66,?,False,False,Do Not Notify,?,?,?,?,code 6,Resolved by 158,12/5/2016 23:51,18/5/2016 00:07
81420,INC0019396,Resolved,True,0,0,6,True,Caller 1580,Opened by 305,14/4/2016 20:42,?,?,Updated by 713,15/4/2016 09:20,Phone,Location 143,Category 53,Subcategory 114,?,?,2 - Medium,2 - Medium,3 - Moderate,Group 66,Resolver 175,False,True,Do Not Notify,?,?,?,?,code 6,Resolved by 158,12/5/2016 23:51,18/5/2016 00:07
81421,INC0019396,Active,True,0,1,7,True,Caller 1580,Opened by 305,14/4/2016 20:42,?,?,Updated by 278,15/4/2016 15:21,Phone,Location 143,Category 53,Subcategory 114,?,?,2 - Medium,2 - Medium,3 - Moderate,Group 66,Resolver 175,False,True,Do Not Notify,?,?,?,?,code 6,Resolved by 158,12/5/2016 23:51,18/5/2016 00:07
81422,INC0019396,Active,True,0,1,9,True,Caller 1580,Opened by 305,14/4/2016 20:42,?,?,Updated by 278,15/4/2016 15:21,Phone,Location 143,Category 53,Subcategory 114,?,?,2 - Medium,2 - Medium,3 - Moderate,Group 66,Resolver 175,False,True,Do Not Notify,?,?,?,?,code 6,Resolved by 158,12/5/2016 23:51,18/5/2016 00:07


,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,sys_created_by,sys_created_at,sys_updated_by,sys_updated_at,contact_type,location,category,subcategory,u_symptom,cmdb_ci,impact,urgency,priority,assignment_group,assigned_to,knowledge,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at
81471,INC0019396,Resolved,True,10,7,63,True,Caller 1580,Opened by 305,14/4/2016 20:42,?,?,Updated by 429,12/5/2016 14:46,Phone,Location 143,Category 53,Subcategory 114,?,?,2 - Medium,2 - Medium,3 - Moderate,Group 70,Resolver 115,False,True,Do Not Notify,?,?,?,?,code 6,Resolved by 158,12/5/2016 23:51,18/5/2016 00:07
81472,INC0019396,Active,True,10,8,64,True,Caller 1580,Opened by 305,14/4/2016 20:42,?,?,Updated by 278,12/5/2016 16:30,Phone,Location 143,Category 53,Subcategory 114,?,?,2 - Medium,2 - Medium,3 - Moderate,Group 70,Resolver 115,False,True,Do Not Notify,?,?,?,?,code 6,Resolved by 158,12/5/2016 23:51,18/5/2016 00:07
81473,INC0019396,Active,True,10,8,66,True,Caller 1580,Opened by 305,14/4/2016 20:42,?,?,Updated by 278,12/5/2016 16:30,Phone,Location 143,Category 53,Subcategory 114,?,?,2 - Medium,2 - Medium,3 - Moderate,Group 70,Resolver 115,False,True,Do Not Notify,?,?,?,?,code 6,Resolved by 158,12/5/2016 23:51,18/5/2016 00:07
81474,INC0019396,Resolved,True,10,8,67,True,Caller 1580,Opened by 305,14/4/2016 20:42,?,?,Updated by 111,12/5/2016 23:51,Phone,Location 143,Category 53,Subcategory 114,?,?,2 - Medium,2 - Medium,3 - Moderate,Group 70,Resolver 26,False,True,Do Not Notify,?,?,?,?,code 6,Resolved by 158,12/5/2016 23:51,18/5/2016 00:07
81475,INC0019396,Closed,False,10,8,68,False,Caller 1580,Opened by 305,14/4/2016 20:42,?,?,Updated by 908,18/5/2016 00:07,Phone,Location 143,Category 53,Subcategory 114,?,?,2 - Medium,2 - Medium,3 - Moderate,Group 70,Resolver 26,False,True,Do Not Notify,?,?,?,?,code 6,Resolved by 158,12/5/2016 23:51,18/5/2016 00:07


In [102]:
display(df.head())
display(df.tail())

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,sys_created_by,sys_created_at,sys_updated_by,sys_updated_at,contact_type,location,category,subcategory,u_symptom,cmdb_ci,impact,urgency,priority,assignment_group,assigned_to,knowledge,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at,target_colum,difference
0,INC0000045,New,True,0,0,0,True,Caller 2403,Opened by 8,2016-02-29 01:16:00,Created by 6,29/2/2016 01:23,Updated by 21,29/2/2016 01:23,Phone,Location 143,Category 55,Subcategory 170,Symptom 72,NaN,2 - Medium,2 - Medium,3 - Moderate,Group 56,NaN,True,False,Do Not Notify,NaN,NaN,NaN,NaN,code 5,Resolved by 149,2016-02-29 11:29:00,5/3/2016 12:00,0,0.0
1,INC0000045,Resolved,True,0,0,2,True,Caller 2403,Opened by 8,2016-02-29 01:16:00,Created by 6,29/2/2016 01:23,Updated by 642,29/2/2016 08:53,Phone,Location 143,Category 55,Subcategory 170,Symptom 72,NaN,2 - Medium,2 - Medium,3 - Moderate,Group 56,NaN,True,False,Do Not Notify,NaN,NaN,NaN,NaN,code 5,Resolved by 149,2016-02-29 11:29:00,5/3/2016 12:00,0,0.0
2,INC0000045,Resolved,True,0,0,3,True,Caller 2403,Opened by 8,2016-02-29 01:16:00,Created by 6,29/2/2016 01:23,Updated by 804,29/2/2016 11:29,Phone,Location 143,Category 55,Subcategory 170,Symptom 72,NaN,2 - Medium,2 - Medium,3 - Moderate,Group 56,NaN,True,False,Do Not Notify,NaN,NaN,NaN,NaN,code 5,Resolved by 149,2016-02-29 11:29:00,5/3/2016 12:00,0,0.0
3,INC0000045,Closed,False,0,0,4,True,Caller 2403,Opened by 8,2016-02-29 01:16:00,Created by 6,29/2/2016 01:23,Updated by 908,5/3/2016 12:00,Phone,Location 143,Category 55,Subcategory 170,Symptom 72,NaN,2 - Medium,2 - Medium,3 - Moderate,Group 56,NaN,True,False,Do Not Notify,NaN,NaN,NaN,NaN,code 5,Resolved by 149,2016-02-29 11:29:00,5/3/2016 12:00,0,0.0
4,INC0000047,New,True,0,0,0,True,Caller 2403,Opened by 397,2016-02-29 04:40:00,Created by 171,29/2/2016 04:57,Updated by 746,29/2/2016 04:57,Phone,Location 165,Category 40,Subcategory 215,Symptom 471,NaN,2 - Medium,2 - Medium,3 - Moderate,Group 70,Resolver 89,True,False,Do Not Notify,NaN,NaN,NaN,NaN,code 5,Resolved by 81,2016-03-01 09:52:00,6/3/2016 10:00,0,1.0


,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,sys_created_by,sys_created_at,sys_updated_by,sys_updated_at,contact_type,location,category,subcategory,u_symptom,cmdb_ci,impact,urgency,priority,assignment_group,assigned_to,knowledge,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at,target_colum,difference
141707,INC0120835,Closed,False,1,0,4,True,Caller 116,Opened by 12,2017-02-16 09:09:00,NaN,NaN,Updated by 27,16/2/2017 09:53,Email,Location 204,Category 42,Subcategory 223,Symptom 494,NaN,2 - Medium,2 - Medium,3 - Moderate,Group 31,Resolver 10,False,True,Do Not Notify,NaN,NaN,NaN,NaN,code 9,Resolved by 9,2017-02-16 09:53:00,16/2/2017 09:53,0,0.0
141708,INC0121064,Active,True,0,0,0,True,Caller 116,Opened by 12,2017-02-16 14:17:00,NaN,NaN,Updated by 908,16/2/2017 14:17,Email,Location 204,Category 42,Subcategory 223,Symptom 494,NaN,2 - Medium,2 - Medium,3 - Moderate,Group 70,Resolver 10,False,False,Do Not Notify,NaN,NaN,NaN,NaN,code 6,Resolved by 9,2017-02-16 16:38:00,16/2/2017 16:38,0,0.0
141709,INC0121064,Active,True,1,0,1,True,Caller 116,Opened by 12,2017-02-16 14:17:00,NaN,NaN,Updated by 60,16/2/2017 15:20,Email,Location 204,Category 42,Subcategory 223,Symptom 494,NaN,2 - Medium,2 - Medium,3 - Moderate,Group 31,NaN,False,False,Do Not Notify,NaN,NaN,NaN,NaN,code 6,Resolved by 9,2017-02-16 16:38:00,16/2/2017 16:38,0,0.0
141710,INC0121064,Resolved,True,1,0,2,True,Caller 116,Opened by 12,2017-02-16 14:17:00,NaN,NaN,Updated by 27,16/2/2017 16:38,Email,Location 204,Category 42,Subcategory 223,Symptom 494,NaN,2 - Medium,2 - Medium,3 - Moderate,Group 31,Resolver 10,False,True,Do Not Notify,NaN,NaN,NaN,NaN,code 6,Resolved by 9,2017-02-16 16:38:00,16/2/2017 16:38,0,0.0
141711,INC0121064,Closed,False,1,0,3,True,Caller 116,Opened by 12,2017-02-16 14:17:00,NaN,NaN,Updated by 27,16/2/2017 16:38,Email,Location 204,Category 42,Subcategory 223,Symptom 494,NaN,2 - Medium,2 - Medium,3 - Moderate,Group 31,Resolver 10,False,True,Do Not Notify,NaN,NaN,NaN,NaN,code 6,Resolved by 9,2017-02-16 16:38:00,16/2/2017 16:38,0,0.0


In [101]:
df.value_counts('active')

active
True     116726
False     24986
Name: count, dtype: int64

In [100]:
df.value_counts('incident_state')

incident_state
Active                38716
New                   36407
Resolved              25751
Closed                24985
Awaiting User Info    14642
Awaiting Vendor         707
Awaiting Problem        461
Awaiting Evidence        38
-100                      5
Name: count, dtype: int64

In [97]:
df.value_counts('opened_by')

opened_by
Opened by  17     41466
Opened by  24      8063
Opened by  131     7269
Opened by  108     6121
Opened by  55      5655
                  ...  
Opened by  447        3
Opened by  80         3
Opened by  440        3
Opened by  486        3
Opened by  144        3
Name: count, Length: 207, dtype: int64

In [4]:
df.columns


Index(['number', 'incident_state', 'active', 'reassignment_count',
       'reopen_count', 'sys_mod_count', 'made_sla', 'caller_id', 'opened_by',
       'opened_at', 'sys_created_by', 'sys_created_at', 'sys_updated_by',
       'sys_updated_at', 'contact_type', 'location', 'category', 'subcategory',
       'u_symptom', 'cmdb_ci', 'impact', 'urgency', 'priority',
       'assignment_group', 'assigned_to', 'knowledge',
       'u_priority_confirmation', 'notify', 'problem_id', 'rfc', 'vendor',
       'caused_by', 'closed_code', 'resolved_by', 'resolved_at', 'closed_at'],
      dtype='object')

In [11]:
df.value_counts('impact')

impact
2 - Medium    134335
3 - Low         3886
1 - High        3491
Name: count, dtype: int64

In [5]:
df.value_counts('incident_state')

incident_state
Active                38716
New                   36407
Resolved              25751
Closed                24985
Awaiting User Info    14642
Awaiting Vendor         707
Awaiting Problem        461
Awaiting Evidence        38
-100                      5
Name: count, dtype: int64

In [7]:
df.value_counts('active')

active
True     116726
False     24986
Name: count, dtype: int64

In [10]:
df.value_counts('reopen_count')

reopen_count
0    139398
1      1918
2       190
3       107
4        48
5        24
6        18
7         5
8         4
Name: count, dtype: int64

In [6]:
df.value_counts('incident_state', 'active')

incident_state
Active                0.273202
New                   0.256908
Resolved              0.181714
Closed                0.176308
Awaiting User Info    0.103322
Awaiting Vendor       0.004989
Awaiting Problem      0.003253
Awaiting Evidence     0.000268
-100                  0.000035
Name: proportion, dtype: float64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141712 entries, 0 to 141711
Data columns (total 36 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   number                   141712 non-null  object
 1   incident_state           141712 non-null  object
 2   active                   141712 non-null  bool  
 3   reassignment_count       141712 non-null  int64 
 4   reopen_count             141712 non-null  int64 
 5   sys_mod_count            141712 non-null  int64 
 6   made_sla                 141712 non-null  bool  
 7   caller_id                141712 non-null  object
 8   opened_by                141712 non-null  object
 9   opened_at                141712 non-null  object
 10  sys_created_by           141712 non-null  object
 11  sys_created_at           141712 non-null  object
 12  sys_updated_by           141712 non-null  object
 13  sys_updated_at           141712 non-null  object
 14  contact_type        

In [15]:
display(df.value_counts('problem_id'))
display(df.value_counts('rfc'))
display(df.value_counts('vendor'))
display(df.value_counts('caused_by'))

problem_id
?                  139417
Problem ID  14        184
Problem ID  2         147
Problem ID  52         82
Problem ID  239        48
                    ...  
Problem ID  181         1
Problem ID  88          1
Problem ID  11          1
Problem ID  195         1
Problem ID  33          1
Name: count, Length: 253, dtype: int64

rfc
?             140721
CHG0000132        20
CHG0001230        20
CHG0000047        18
CHG0001656        17
               ...  
CHG0000207         2
CHG0001829         1
CHG0000530         1
CHG0001437         1
CHG0001720         1
Name: count, Length: 182, dtype: int64

vendor
?           141468
code 8s        167
Vendor 1        69
Vendor 3         6
Vendor 2         2
Name: count, dtype: int64

caused_by
?             141689
CHG0000097        11
CHG0000132         7
CHG0001327         5
Name: count, dtype: int64

In [ ]:
display(df.value_counts('opened_at'))
display(df.value_counts('resolved_at'))

In [99]:
df['resolved_at'].isnull().value_counts()

resolved_at
False    138571
True       3141
Name: count, dtype: int64

In [ ]:
df['resolved_at'].value_counts()

In [ ]:
df_nan.head()

In [ ]:
check_dates =df_nan[['opened_at','resolved_at']]
check_dates.head(50)

In [ ]:
df_nan.info()

In [ ]:
#This drops all columns with Nan values don't know what we would use this for
df_drop = df_nan.dropna(axis = 1).copy()
df_drop.info()

In [ ]:
#need to convert all strings to numbers before this code will work
df_nan.corr()['target'].sort_values()